# Using the EconModelClass

**Table of contents**<a id='toc0_'></a>    
- 1. [Baseline Usage](#toc1_)    
- 2. [Copy, save and load](#toc2_)    
- 3. [To and from dictionary](#toc3_)    
- 4. [Numba](#toc4_)    
  - 4.1. [Advanced: Recompilling](#toc4_1_)    
  - 4.2. [Advanced: Type inference](#toc4_2_)    
  - 4.3. [Advanced: Error handling](#toc4_3_)    
- 5. [Calling C++](#toc5_)    
- 6. [Clean-up](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

`EconModel` is a small package for easily working with economic models in Python. It has four objectives:

1. Make it easy to write well-structed code.
1. Provide standard functionality for copying, saving and loading.
1. Provide an easy interface to call [numba](http://numba.pydata.org/) JIT compilled functions.
1. Provide an easy interface to call C++ functions.

The two first objectives should be of interest to everybody, while the two last objectives are for more advanced users.

This notebook provides a simple example for using the `EconModelClass`. See documentation in [EconModel/README.md](https://github.com/NumEconCopenhagen/EconModel#econmodel).

In [1]:
%load_ext autoreload
%autoreload 2

import shutil
import numpy as np
import numba as nb

from EconModel import EconModelClass, jit

## 1. <a id='toc1_'></a>[Baseline Usage](#toc0_)

The model is **required** to have the following three methods:

1. `.settings()`: Choose fundamental settings.
1. `.setup()`: Set free parameters.
1. `.allocate()`: Set compound parameters and allocate arrays.

When the model is initialized the call order is:

1. `.settings()`
2. `.setup()` 
3. `.allocate()`

Per default the namespaces `.par` (for parameters), `.sol` (for solution objects) and `.sim` (for simulation objects) are available.

The philosophy of the package is that all the content of the model is contained in these namespaces. 

After the model has been initialized, no additional entries should be added to the namespaces, and each entry should only change *value(s)* (or shape for arrays), but never *type* (or number of dimensions for arrays).

In [2]:
class MyModelClass(EconModelClass):
    
    # __init__ is inherited from EconModelClass
    
    def settings(self): # required
        """ choose settings """
            
        pass # nothing chosen here
    
    def setup(self): # required
        """ set free parameters """
        
        par = self.par
        
        par.N = 10
        par.a = 2.0
        par.b = 1.0
        par.threads = 4
        par.txt = 'a'
        par.txtlist = 'N|threads'
        
    def allocate(self): # required
        """ set compound parameters and allocate arrays """
        
        par = self.par
        sol = self.sol
        
        par.X = np.linspace(0,10,par.N)
        sol.Y = np.zeros(par.N)
    
    def solve(self): # user-defined
        """ solve the model"""
        
        par = self.par
        sol = self.sol
        
        sol.Y[:] = par.X*(par.a+par.b)


In `.settings()` the user can change the default behavior by specifying: 

1. `self.savefolder = str`: Filepath to save in and load from (default: 'saved').
1. `self.namespaces = [str]`: List of namespaces (in addition to `.par`, `.sol`, `.sol`).
1. `self.other_attrs = [str]`: List of additional attributes to be copied and saved.


In [3]:
model = MyModelClass(name='example')

In [4]:
model.solve()

**Print description:**

In [5]:
print(model)

Modelclass: MyModelClass
Name: example

namespaces: ['par', 'sol', 'sim']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray[float64] with shape = (10,) [0.0 mb]
 memory, gb: 0.0

sol:
 Y = ndarray[float64] with shape = (10,) [0.0 mb]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



**Updating at initialization.** Called after `.setup()`, but before `.allocate()`.

In [6]:
model_alt = MyModelClass(name='alt',par={'a':3.0,'b':1.2})
print(model_alt.par.a)

3.0


**Unpack for later use:**

In [7]:
par = model.par
sol = model.sol

**Under the hood:** Each namespace is techically a `SimpleNamespace` (from `types`)

In [8]:
type(par)

types.SimpleNamespace

## 2. <a id='toc2_'></a>[Copy, save and load](#toc0_)

In [9]:
model_copy = model.copy(name='example_copy')
model_copy.save()

In [10]:
model_loaded = MyModelClass(name='example_copy',load=True)
print(model_loaded)

Modelclass: MyModelClass
Name: example_copy

namespaces: ['par', 'sol', 'sim']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray[float64] with shape = (10,) [0.0 mb]
 memory, gb: 0.0

sol:
 Y = ndarray[float64] with shape = (10,) [0.0 mb]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



## 3. <a id='toc3_'></a>[To and from dictionary](#toc0_)

In [11]:
modeldict = model.as_dict()
model_dict = MyModelClass(name='example_dict',from_dict=modeldict)
print(model_dict)

Modelclass: MyModelClass
Name: example_dict

namespaces: ['par', 'sol', 'sim']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray[float64] with shape = (10,) [0.0 mb]
 memory, gb: 0.0

sol:
 Y = ndarray[float64] with shape = (10,) [0.0 mb]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



## 4. <a id='toc4_'></a>[Numba](#toc0_)

**Goal:** Call [numba](http://numba.pydata.org/) JIT compilled functions easily using namespaces.

**Problem:** [numba](http://numba.pydata.org/) only allows specific types (and not e.g. `SimpleNamespace`).

**Under-the-hood:** `jit(model)` temporarily turns all namespaces into `namedtuples`, which can be used as input in JIT compilled functions. 

In [12]:
@nb.njit
def fun(par,sol):
    sol.Y[:] = par.X*(par.a+par.b)

In [13]:
def test_numba(model):
    with jit(model) as model_jit:
        fun(model_jit.par,model_jit.sol)

In [14]:
%time test_numba(model)
%time test_numba(model)

CPU times: total: 656 ms
Wall time: 1.54 s
CPU times: total: 0 ns
Wall time: 997 µs


**Note:** The first run is slower due to compiling.

**Check result:**

In [15]:
assert np.allclose(par.X*(par.a+par.b),sol.Y)

### 4.1. <a id='toc4_1_'></a>[Advanced: Recompilling](#toc0_)

When a new model is created, re-compilation of JIT functions are normally necessary:

In [16]:
model_new = MyModelClass(name='numba_new')
%time test_numba(model_new)
%time test_numba(model_new)

CPU times: total: 172 ms
Wall time: 166 ms
CPU times: total: 0 ns
Wall time: 0 ns


This is *not* the case, when a copy is made instead:

In [17]:
model_new_ = model_new.copy(name='numba_new_copy')
%time test_numba(model_new_)

CPU times: total: 15.6 ms
Wall time: 998 µs


### 4.2. <a id='toc4_2_'></a>[Advanced: Type inference](#toc0_)

When the model is created **all types are infered** and the information is saved in the nested-dict `_ns_specs`. E.g.:

In [18]:
model._ns_specs['types']

{'par': {'N': [int, numpy.int32],
  'a': [float, numpy.float64],
  'b': [float, numpy.float64],
  'threads': [int, numpy.int32],
  'txt': [str],
  'txtlist': [str],
  'X': [numpy.ndarray]},
 'sol': {'Y': [numpy.ndarray]},
 'sim': {}}

In [19]:
model._ns_specs['np_dtypes']

{'par': {'X': dtype('float64')}, 'sol': {'Y': dtype('float64')}, 'sim': {}}

In [20]:
model._ns_specs['np_ndims']

{'par': {'X': 1}, 'sol': {'Y': 1}, 'sim': {}}

**The types must NOT change afterwards**. If they do, an error is raised when using  `jit`.

In [21]:
def test_numba_errors(model):
    try:
        test_numba(model)
    except ValueError as e:
        print(f'ValueError: {e}')
    else:
        print('no errors raised')

In [22]:
model_test = model.copy()
model_test.par.a = 1
test_numba_errors(model_test)

ValueError: par.a has type <class 'int'>, should be in [<class 'float'>, <class 'numpy.float64'>]


In [23]:
model_test = model.copy()
model_test.par.c = np.nan
test_numba_errors(model_test)

ValueError: c is not allowed in .par


In [24]:
model_test = model.copy()
model_test.par.X = np.zeros((1,1))
test_numba_errors(model_test)

ValueError: par.X has ndim 2, should be 1


In [25]:
model_test = model.copy()
model_test.par.X = model.par.X.astype('int')
test_numba_errors(model_test)

ValueError: par.X has dtype int32, should be float64


**If types *must* to be changed, then it is necessary to re-infer types:**

In [26]:
model_test = model.copy()
model_test.par.a = 1
model_test.infer_types()
test_numba_errors(model_test)

no errors raised


**Namedtuples** are immutables, so attributes cannot be changed while using `jit`:

In [27]:
try:
    with jit(model) as model_jit:
        model_jit.par.a = 3.0
except AttributeError as e:
    pass

Traceback (most recent call last):
  File "C:\Users\gmf123\AppData\Local\Temp\ipykernel_18836\1634112854.py", line 3, in <module>
    model_jit.par.a = 3.0
    ^^^^^^^^^^^^^^^
AttributeError: can't set attribute


### 4.3. <a id='toc4_3_'></a>[Advanced: Error handling](#toc0_)

Printing of exceptions can be turned off if necessary.

In [28]:
try:
    with jit(model,show_exc=False) as model_jit:
        model_jit.par.a = 3.0
except AttributeError as e:
    print(e)

can't set attribute


## 5. <a id='toc5_'></a>[Calling C++](#toc0_)

**Goal:** Call C++ functions easily using namespaces.

**Problems:** Requires compilation and linking.

**Compilers:** One of these compilers must be installed

* **vs**: Free *Microsoft Visual Studio 2022 Community Edition* ([link](https://visualstudio.microsoft.com/downloads/))
* **intel:** Free *Intel one API* ([link](https://www.intel.com/content/www/us/en/developer/tools/oneapi/base-toolkit-download.html))

For parallization we will use **OpenMP**.

**Platform:** Tested on *Windows*.

**Interface:**

1. all elements in **namespaces** are availible as **C++ structs** with `_struct` appended (default, else specify `.cpp_structsmap` in `.settings()`).
2. `cpptools.link_to_cpp()` is called by `.link_to_cpp` using `.cpp_filename` and the result is returned in `.cpp`.
3. `cpp_options` is the compiler options.
4. by default `force_compile=True` when calling `.link_to_cpp`.

**Details:** Provided in `02. More on calling C++.ipynb`.

**The extended model specification is:**

In [29]:
class MyModelClass_cpp(MyModelClass):
    
    def settings(self):
        
        self.cpp_filename = 'cppfuncs/example.cpp' # required for using C++
        self.cpp_options = {'compiler':'vs'} # optional
        self.cpp_structsmap = {'par':'par_struct'} # optional
    
model_cpp = MyModelClass_cpp(name='model_cpp') 

**Link:**

In [30]:
model_cpp.link_to_cpp(do_print=True)

Linking to: cppfuncs/example.cpp

### finding all included files ###

logs.cpp
par_struct.cpp
sol_struct.cpp
example_include.cpp

### writing structs ###

cppfuncs/par_struct.cpp
 int N;
 double a;
 double b;
 int threads;
 char* txt;
 char* txtlist;
 double* X;

cppfuncs/sol_struct.cpp
 double* Y;

cppfuncs/sim_struct.cpp

### analyzing cppfuncs/example.cpp ###

function: fun
return type: void
argument types: ['par_struct*', 'sol_struct*']

function: fun_nostruct
return type: void
argument types: ['double*', 'double*', 'int', 'double', 'double', 'int', 'char*']

### analyzing cppfuncs/logs.cpp ###

### analyzing cppfuncs/par_struct.cpp ###

### analyzing cppfuncs/sol_struct.cpp ###

### analyzing cppfuncs/example_include.cpp ###

function: fun_nostruct_alt
return type: void
argument types: ['double*', 'double*', 'int', 'double', 'double', 'int', 'char*']

function: I_DO_NOTHING
return type: double
argument types: []

### compiling and linking ###

compile.bat:
cd /d "C:/Program Files/

terminal:

(base) c:\Users\gmf123\Dropbox\Repositories\EconModelNotebooks>cd /d "C:/Program Files/Microsoft Visual Studio/2022/Community/VC/Auxiliary/Build/" 

(base) C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Auxiliary\Build>call vcvarsall.bat x64 
**********************************************************************
** Visual Studio 2022 Developer Command Prompt v17.0.6
** Copyright (c) 2021 Microsoft Corporation
**********************************************************************
[vcvarsall.bat] Environment initialized for: 'x64'
example.cpp
setup_omp.cpp
Generating Code...
Microsoft (R) Incremental Linker Version 14.30.30709.0
Copyright (C) Microsoft Corporation.  All rights reserved.

/out:example.dll 
/dll 
/implib:example.lib 
example.obj 
setup_omp.obj 
   Creating library example.lib and object example.exp

Microsoft (R) C/C++ Optimizing Compiler Version 19.30.30709 for x64
Copyright (C) Microsoft Corporation.  All rights reserved.


C++ files compiled
C++ f

**Compile warnings and errors** are shown in the terminal.

**Call C++:** It is now easy to call the C++ function

In [31]:
model_cpp.cpp.fun(par,sol)
assert np.allclose(par.X*(par.a+par.b),sol.Y)


fun(...)
omp_get_thread_num() =  1, omp_get_num_procs() =  8
string-value: a
looked up value par->a = 2
is "a" in "N|threads": false
looked up value par->N = 10
looked up value par->threads = 4



**Re-compile:**

In [32]:
model_cpp.cpp.recompile()

**Multiple models:** 

In [33]:
# a. don't compile when the .dll file is already loaded
model_cpp_alt = MyModelClass_cpp('testcpp_alt')
model_cpp_alt.link_to_cpp(force_compile=False) # default is force_compile = True

# b. function call
par = model_cpp_alt.par
model_cpp_alt.cpp.fun_nostruct(par.X,sol.Y,par.N,par.a,par.b,par.threads,par.txt)
assert np.allclose(par.X*(par.a+par.b),sol.Y)

# c. delink without removing the .dll file
model_cpp.cpp.delink()

# d. delete model -> automatically calls .cpp.delink()
del model_cpp_alt

# e. now we can compile again
model_cpp_alt = MyModelClass_cpp('testcpp_alt')
model_cpp_alt.link_to_cpp()
model_cpp_alt.cpp.clean_up()


fun_nostruct(...)
omp_get_thread_num() =  0, omp_get_num_procs() =  8
string-value: a
test was succesfull



## 6. <a id='toc6_'></a>[Clean-up](#toc0_)

In [34]:
shutil.rmtree('saved')